# Task 4: EMail Spam Detection with ML
# Batch: OIBSP May P1
# By Ahan Bhatt

In [1]:
import numpy as np;
import pandas as pd;
from nltk.corpus import stopwords
import re
from nltk.stem import SnowballStemmer
from sklearn.linear_model import LogisticRegression,LinearRegression,SGDClassifier;
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder;
from sklearn.model_selection import train_test_split;

In [3]:
text = pd.read_csv('C:/Users/HP/Documents/Indus/Internship/Oasis Infobyte/Tasks/Task 4/spam.csv', encoding = 'latin-1')
text.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
text.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace = True)
text.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
text.columns = ['Category', 'Message']

In [6]:
text.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
x = text['Message'].iloc[2]
x

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [8]:
x.split(' ')

['Free',
 'entry',
 'in',
 '2',
 'a',
 'wkly',
 'comp',
 'to',
 'win',
 'FA',
 'Cup',
 'final',
 'tkts',
 '21st',
 'May',
 '2005.',
 'Text',
 'FA',
 'to',
 '87121',
 'to',
 'receive',
 'entry',
 'question(std',
 'txt',
 "rate)T&C's",
 'apply',
 "08452810075over18's"]

In [9]:
len(x.split(' '))

28

In [10]:
x.split(' ')[4]

'a'

In [11]:
text['word_count'] = text['Message'].agg(lambda x: len(x.split(' ')))
text.head()

,Category,Message,word_count
0,ham,"Go until jurong point, crazy.. Available only ...",20
1,ham,Ok lar... Joking wif u oni...,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28
3,ham,U dun say so early hor... U c already then say...,11
4,ham,"Nah I don't think he goes to usf, he lives aro...",13


In [12]:
text.groupby('Category').agg(['mean', 'std'])

C:\Users\HP\AppData\Local\Temp\ipykernel_17240\1468315363.py:1: FutureWarning: ['Message'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  text.groupby('Category').agg(['mean', 'std'])


word_count           
               mean        std
Category                      
ham       14.323938  11.538966
spam      23.892905   5.826977

In [13]:
text['char_count'] = text['Message'].agg(len)
text.head()

,Category,Message,word_count,char_count
0,ham,"Go until jurong point, crazy.. Available only ...",20,111
1,ham,Ok lar... Joking wif u oni...,6,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28,155
3,ham,U dun say so early hor... U c already then say...,11,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",13,61


In [14]:

text.groupby('Category').agg(['mean', 'std'])

C:\Users\HP\AppData\Local\Temp\ipykernel_17240\2387802731.py:1: FutureWarning: ['Message'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  text.groupby('Category').agg(['mean', 'std'])


word_count             char_count           
               mean        std        mean        std
Category                                             
ham       14.323938  11.538966   71.023627  58.016023
spam      23.892905   5.826977  138.866131  29.183082

In [16]:

import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
stop

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [17]:
stop.append('u')
stop.append('ur')
stop.append('2') # to
stop.append('4') # for
stop.append('n') # and

In [18]:
text['Message'] = text['Message'].agg(lambda x: x.lower())
text.head()

,Category,Message,word_count,char_count
0,ham,"go until jurong point, crazy.. available only ...",20,111
1,ham,ok lar... joking wif u oni...,6,29
2,spam,free entry in 2 a wkly comp to win fa cup fina...,28,155
3,ham,u dun say so early hor... u c already then say...,11,49
4,ham,"nah i don't think he goes to usf, he lives aro...",13,61


In [19]:
text['n_of_stupwords'] = text['Message'].agg(lambda x: len([w for w in x.split() if w in stop]))
text.head()

,Category,Message,word_count,char_count,n_of_stupwords
0,ham,"go until jurong point, crazy.. available only ...",20,111,5
1,ham,ok lar... joking wif u oni...,6,29,1
2,spam,free entry in 2 a wkly comp to win fa cup fina...,28,155,6
3,ham,u dun say so early hor... u c already then say...,11,49,4
4,ham,"nah i don't think he goes to usf, he lives aro...",13,61,6


In [20]:
[w for w in x.split() if w in stop] 

['in', '2', 'a', 'to', 'to', 'to']

In [21]:
text.groupby('Category').agg(['mean'])

C:\Users\HP\AppData\Local\Temp\ipykernel_17240\496926412.py:1: FutureWarning: ['Message'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  text.groupby('Category').agg(['mean'])


,word_count,char_count,n_of_stupwords
,mean,mean,mean
Category,,,
ham,14.323938,71.023627,5.846010
spam,23.892905,138.866131,7.116466


In [22]:
#Remove all charactes
'''
ex1='Hello. Can I ask you a question?! That will cost me 100$, but it is OK.'
Regular expression is a powerful package to manage strings.
\:means a special sequence. 
\d:retuns a matche where a string contains a digit. 
\s:returns a match where the string contains white space
re.findall('\s',ex1)
\w:returns all word and numeric characters 
re.findall('\w',ex1)
len(re.findall('\w',ex1))
+:returns all characters' occurances
re.findall('l+',ex1)
[hi?]:returns matches to each of the characters inside [] 
re.findall('[hi?]',ex1)
Combine some commands:
re.findall('\w\s',ex1)#Find word characters followed by a white space
re.findall('\d+\$',ex1)
[^hi]:return all characters except those followed by ^
re.findall('[^hi?]',ex1)
re.findall('[^hi?\s]',ex1)
re.findall('[\w\s]',ex1)
re.findall('[^\w\s]',ex1)
re.sub:finds a pattern and substitute it with a replacement
re.sub('will','might',ex1)
re.sub('[^\w\s]',"",ex1)
'''


'\nex1=\'Hello. Can I ask you a question?! That will cost me 100$, but it is OK.\'\nRegular expression is a powerful package to manage strings.\n\\:means a special sequence. \n\\d:retuns a matche where a string contains a digit. \n\\s:returns a match where the string contains white space\nre.findall(\'\\s\',ex1)\n\\w:returns all word and numeric characters \nre.findall(\'\\w\',ex1)\nlen(re.findall(\'\\w\',ex1))\n+:returns all characters\' occurances\nre.findall(\'l+\',ex1)\n[hi?]:returns matches to each of the characters inside [] \nre.findall(\'[hi?]\',ex1)\nCombine some commands:\nre.findall(\'\\w\\s\',ex1)#Find word characters followed by a white space\nre.findall(\'\\d+\\$\',ex1)\n[^hi]:return all characters except those followed by ^\nre.findall(\'[^hi?]\',ex1)\nre.findall(\'[^hi?\\s]\',ex1)\nre.findall(\'[\\w\\s]\',ex1)\nre.findall(\'[^\\w\\s]\',ex1)\nre.sub:finds a pattern and substitute it with a replacement\nre.sub(\'will\',\'might\',ex1)\nre.sub(\'[^\\w\\s]\',"",ex1)\n'

In [23]:
ex1 = 'Hello. Can I ask you a question?! That willl cost me 100$, but it is OK.'
ex1

'Hello. Can I ask you a question?! That willl cost me 100$, but it is OK.'

In [24]:
re.findall('l', ex1)

['l', 'l', 'l', 'l', 'l']

In [25]:
re.findall('hello', ex1.lower())

['hello']

In [26]:
re.findall('\s', ex1) # Space

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

In [27]:
re.findall('l+', ex1)

['ll', 'lll']

In [28]:
re.findall('\w', ex1)

['H',
 'e',
 'l',
 'l',
 'o',
 'C',
 'a',
 'n',
 'I',
 'a',
 's',
 'k',
 'y',
 'o',
 'u',
 'a',
 'q',
 'u',
 'e',
 's',
 't',
 'i',
 'o',
 'n',
 'T',
 'h',
 'a',
 't',
 'w',
 'i',
 'l',
 'l',
 'l',
 'c',
 'o',
 's',
 't',
 'm',
 'e',
 '1',
 '0',
 '0',
 'b',
 'u',
 't',
 'i',
 't',
 'i',
 's',
 'O',
 'K']

In [29]:
re.findall('\w+', ex1)

['Hello',
 'Can',
 'I',
 'ask',
 'you',
 'a',
 'question',
 'That',
 'willl',
 'cost',
 'me',
 '100',
 'but',
 'it',
 'is',
 'OK']

In [30]:
re.findall('\d', ex1)

['1', '0', '0']

In [31]:
re.findall('\d+', ex1)

['100']

In [32]:
ex1

'Hello. Can I ask you a question?! That willl cost me 100$, but it is OK.'

In [33]:
re.findall('\w\s', ex1)

['n ', 'I ', 'k ', 'u ', 'a ', 't ', 'l ', 't ', 'e ', 't ', 't ', 's ']

In [34]:
re.findall('\w+\s', ex1)

['Can ',
 'I ',
 'ask ',
 'you ',
 'a ',
 'That ',
 'willl ',
 'cost ',
 'me ',
 'but ',
 'it ',
 'is ']

In [35]:
re.findall('\d+\$', ex1)

['100$']

In [36]:
re.findall('\w+\.\s\w+', ex1)

['Hello. Can']

In [37]:
# w3school

In [38]:
re.findall('\w+\.\s\w+', ex1)

['Hello. Can']

In [39]:
re.findall('[\w\s]', ex1)

['H',
 'e',
 'l',
 'l',
 'o',
 ' ',
 'C',
 'a',
 'n',
 ' ',
 'I',
 ' ',
 'a',
 's',
 'k',
 ' ',
 'y',
 'o',
 'u',
 ' ',
 'a',
 ' ',
 'q',
 'u',
 'e',
 's',
 't',
 'i',
 'o',
 'n',
 ' ',
 'T',
 'h',
 'a',
 't',
 ' ',
 'w',
 'i',
 'l',
 'l',
 'l',
 ' ',
 'c',
 'o',
 's',
 't',
 ' ',
 'm',
 'e',
 ' ',
 '1',
 '0',
 '0',
 ' ',
 'b',
 'u',
 't',
 ' ',
 'i',
 't',
 ' ',
 'i',
 's',
 ' ',
 'O',
 'K']

In [40]:
re.findall('[^hi?]', ex1)

['H',
 'e',
 'l',
 'l',
 'o',
 '.',
 ' ',
 'C',
 'a',
 'n',
 ' ',
 'I',
 ' ',
 'a',
 's',
 'k',
 ' ',
 'y',
 'o',
 'u',
 ' ',
 'a',
 ' ',
 'q',
 'u',
 'e',
 's',
 't',
 'o',
 'n',
 '!',
 ' ',
 'T',
 'a',
 't',
 ' ',
 'w',
 'l',
 'l',
 'l',
 ' ',
 'c',
 'o',
 's',
 't',
 ' ',
 'm',
 'e',
 ' ',
 '1',
 '0',
 '0',
 '$',
 ',',
 ' ',
 'b',
 'u',
 't',
 ' ',
 't',
 ' ',
 's',
 ' ',
 'O',
 'K',
 '.']

In [41]:
re.findall('[^\w\s]', ex1)

['.', '?', '!', '$', ',', '.']

In [42]:
re.sub('willl', 'might', ex1)

'Hello. Can I ask you a question?! That might cost me 100$, but it is OK.'

In [43]:
text['n_of_signs'] = text['Message'].agg(lambda x: len(re.findall('[^\w\s]', x)))
text.head(2)

,Category,Message,word_count,char_count,n_of_stupwords,n_of_signs
0,ham,"go until jurong point, crazy.. available only ...",20,111,5,9
1,ham,ok lar... joking wif u oni...,6,29,1,6


In [44]:
text['Message'] = text['Message'].agg(lambda x: re.sub('[^\w\s]', '', x))
text.head()

,Category,Message,word_count,char_count,n_of_stupwords,n_of_signs
0,ham,go until jurong point crazy available only in ...,20,111,5,9
1,ham,ok lar joking wif u oni,6,29,1,6
2,spam,free entry in 2 a wkly comp to win fa cup fina...,28,155,6,6
3,ham,u dun say so early hor u c already then say,11,49,4,6
4,ham,nah i dont think he goes to usf he lives aroun...,13,61,6,2


In [45]:
# Stemming all words 
# Predict, Predicting, Prediction

In [46]:
stemmer = SnowballStemmer('english')
stemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')